In [13]:
import os
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/anandreddy.s3215/DataScience_Project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="anandreddy.s3215"
os.environ['MLFLOW_TRACKING_PASSWORD']=os.getenv("MLFLOW_TRACKING_PASSWORD")

In [3]:
import os
%pwd

'd:\\MAIN_PROJECTS\\DataScience_Project\\research'

In [4]:
os.chdir("../")
%pwd

'd:\\MAIN_PROJECTS\\DataScience_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: str
    all_params: dict
    target_column: str
    mlflow_uri: str

In [6]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml,create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri=os.environ['MLFLOW_TRACKING_URI']
        )
        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            
            (rmse,mae,r2) = self.eval_metrics(actual=test_y,pred=predicted_qualities)
            
            scores = {"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model")
            else:
                mlflow.sklearn.log_model(model,"model")
            

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-23 13:16:55,169: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-23 13:16:55,171: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-23 13:16:55,173: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-23 13:16:55,176: INFO: common: Created directory at: artifacts]
[2025-06-23 13:16:55,176: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-06-23 13:16:55,931: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


In [1]:
import mlflow
print(mlflow.__version__)

2.11.3
